In [1]:
import krpc
import time
from math import e
import numpy as np
import matplotlib.pyplot as plt

In [2]:
conn = krpc.connect(name='Auto Mun')
vessel = conn.space_center.active_vessel

**Set Up Planet Objects**

In [3]:
for name, obj in conn.space_center.bodies.items():
    if name == 'Kerbin':
        kerbin = obj
    elif name == 'Mun':
        mun = obj

mass_kerbin = 5.2915158e22  # (kg)
mass_mun = 9.7599066e20     # (kg)

G = 6.67408e-11  # (m**3/kg/s**2) gravitational constant

**Reference Frames**

In [4]:
kerbin_nrrf = kerbin.non_rotating_reference_frame

**Helper Functions**

In [5]:
def warp_to_node(conn, vessel, node, burn_duration):
    # set attitude
    vessel.control.sas_mode = conn.space_center.SASMode.maneuver    
    ang_vel = vessel.angular_velocity(vessel.orbit.body.reference_frame)
    oriented = all(ang_vel[i] < 0.01 for i in range(3))
    while not oriented:
        ang_vel = vessel.angular_velocity(vessel.orbit.body.reference_frame)
        oriented = all(ang_vel[i] < 0.01 for i in range(3))
    
    # warp
    end_warp = node.ut - burn_duration
    conn.space_center.rails_warp_factor = 4
    while conn.space_center.ut < (end_warp - 1000):
        time.sleep(0.1)
    conn.space_center.rails_warp_factor = 3
    while conn.space_center.ut < (end_warp - 300):
        time.sleep(0.1)
    conn.space_center.rails_warp_factor = 2
    while conn.space_center.ut < (end_warp - 100):
        time.sleep(0.1)
    conn.space_center.rails_warp_factor = 1
    while conn.space_center.ut < (end_warp - 15):
        time.sleep(0.1)
    conn.space_center.rails_warp_factor = 0
    
    
def execute_node(conn, vessel, node, burn_duration):
    while conn.space_center.ut < (node.ut-(burn_duration/2)):
        time.sleep(0.1)

    while node.remaining_delta_v > 15:
        vessel.control.throttle = 1
    while node.remaining_delta_v > 0.1:
        vessel.control.throttle = 0.05

    node.remove()
    vessel.control.throttle = 0

**Orbital Mechanics**

In [6]:
def calc_burn_duration(vessel, dV):
    ''' Calculates the burn time for a given vessel and a given delta-V. '''
    m = vessel.mass
    isp = vessel.specific_impulse
    thrust = vessel.available_thrust
    term1 = (m*isp*9.81)/(thrust)
    term2 = (1-e**(dV/(isp*9.81)))
    return abs(term1*term2)


def calc_circ_orb_speed(r, M):
    ''' Calculates the orbital speed of a vessel in a circular orbit. '''
    return np.sqrt(G*M/r)


def vis_viva(r, a, M):
    '''
    Calculates the orbital speed of a vessel at some point in an
    elliptical orbit given the semi-major axis and mass of orbiting
    body.
    '''
    return np.sqrt(G*M*((2/r)-(1/a)))


def orbital_period(a, M):
    ''' 
    Calculates the orbital period of a satellite given
    the semi-major axis of the orbit and the mass of the
    orbiting body.
    '''
    mu = G*M
    return (2*np.pi)*np.sqrt(a**3/mu)


def v_pe_hyperbolic(M, a, e):
    ''' Find periapsis speed of hyperbolic orbit. '''
    k = G*M  # gravitational parameter
    return ((-k/a)*(1+e)/(e-1))**0.5

**Kerbin Ascent**

This ascent profile is custom for this vehicle. It would be nice to have a generalized script (at least attempt) to launch any vehicle to orbit.

`TODO`: Convert times to use `MET` rather than `time.time()` so that physical time warp is stable.

In [7]:
# vehicle telemetry
telem = vessel.flight(vessel.orbit.body.reference_frame)

# go straight up at first
vessel.auto_pilot.target_pitch_and_heading(90, 90)
vessel.auto_pilot.engage()
time.sleep(1)

# throttle 100% and ignition
vessel.control.throttle = 1
vessel.control.activate_next_stage()

# wait until going 100 m/s straight up
v_speed = telem.vertical_speed
while v_speed < 100:
    v_speed = telem.vertical_speed
    time.sleep(0.1)

# pitch over at a rate of 1.125 deg/s for 40 s
# this gets vehicle to 45 deg by 10 km
pitch_start = time.time()
pitch_end = pitch_start + 40  # 40 seconds of pitching
time_to_deg = 1.125
while time.time() < pitch_end:
    tgt_deg = 90 - ((time.time() - pitch_start) * time_to_deg)
    vessel.auto_pilot.target_pitch_and_heading(tgt_deg, 90)
    
print('Lower atmosphere pitch complete.')
    
# pitch over at a lower rate until first stage depleted
pitch_start = time.time()
pitch_end = pitch_start + 25  # ~25 seconds until depleted
while time.time() < pitch_end:
    tgt_deg = 45 - ((time.time() - pitch_start) * 0.4)
    vessel.auto_pilot.target_pitch_and_heading(tgt_deg, 90)

# MECO
stg_1_resrcs = vessel.resources_in_decouple_stage(stage=9, cumulative=False)
stg_1_lqd_fu = stg_1_resrcs.amount('LiquidFuel')
while stg_1_lqd_fu > 0.01:
    stg_1_lqd_fu = stg_1_resrcs.amount('LiquidFuel')
    vessel.auto_pilot.target_pitch_and_heading(35, 90)
    time.sleep(1)
    
print('MECO.')
    
# S1 Sep
vessel.control.throttle = 0
vessel.control.activate_next_stage()
time.sleep(1)

print('Stage 1 sep.')

# S2 Ignition
vessel.control.activate_next_stage()
vessel.control.throttle = 1

# pitch over slowly on S2 until apoapsis ~100 km
pitch_start = time.time()
pitch_end = pitch_start + 25
while time.time() < pitch_end:
    tgt_deg = 35 - ((time.time() - pitch_start) * 1.0)
    vessel.auto_pilot.target_pitch_and_heading(tgt_deg, 90)
    
apo_alt = vessel.orbit.apoapsis_altitude
while apo_alt < 100000:
    apo_alt = vessel.orbit.apoapsis_altitude
    vessel.auto_pilot.target_pitch_and_heading(10, 90)
    
# stabilize for coast to apoapsis
vessel.control.throttle = 0
print('SECO.')
vessel.auto_pilot.disengage()
time.sleep(1)
vessel.control.sas = True
time.sleep(1)
vessel.control.sas_mode = conn.space_center.SASMode.prograde
time.sleep(1)

Lower atmosphere pitch complete.
MECO.
Stage 1 sep.
SECO.


**Circularize Kerbin Orbit**

In [ ]:
# wait until out of atmosphere
conn.space_center.physics_warp_factor = 3
while vessel.flight().mean_altitude < 70000:
    time.sleep(1)
    
conn.space_center.physics_warp_factor = 0
    
# fairing & LES sep
time.sleep(3)
vessel.control.activate_next_stage()

print('Fairing sep.')
print('LAS sep.')
    
print('Out of atmosphere.')

# go to map screen
time.sleep(3)
conn.space_center.camera.mode = conn.space_center.CameraMode.map

# relevant times
time.sleep(1)
ut_set = conn.space_center.ut
t_to_ap = vessel.orbit.time_to_apoapsis
ut_ap = ut_set + t_to_ap

# determine delta-V required to circularize
V_ap_reqd = calc_circ_orb_speed(r=vessel.orbit.apoapsis, M=mass_kerbin)
V_ap_curr = vis_viva(r=vessel.orbit.apoapsis, a=vessel.orbit.semi_major_axis, \
                     M=mass_kerbin)
dV_node = abs(V_ap_reqd - V_ap_curr)

# create circularization node
circ = vessel.control.add_node(ut=ut_ap)
circ.prograde = dV_node

print('Created circularization maneuver node.')

# set SAS to maneuver and wait until oriented
print('Orienting for maneuver.')

vessel.control.sas_mode = conn.space_center.SASMode.maneuver
time.sleep(3)
ang_vel = vessel.angular_velocity(vessel.orbit.body.reference_frame)
oriented = all(ang_vel[i] < 0.01 for i in range(3))
while not oriented:
    ang_vel = vessel.angular_velocity(vessel.orbit.body.reference_frame)
    oriented = all(ang_vel[i] < 0.01 for i in range(3))

# time warp to maneuver
print('Warping to maneuver.')

burn_time = calc_burn_duration(vessel, dV_node)
conn.space_center.physics_warp_factor = 3
while conn.space_center.ut < (ut_ap - (burn_time / 2) - 15):
    time.sleep(1)
    
conn.space_center.physics_warp_factor = 0
conn.space_center.camera.mode = conn.space_center.CameraMode.automatic
time.sleep(1)

# execute maneuver
while conn.space_center.ut < (ut_ap - (burn_time / 2)):
    time.sleep(0.1)

while circ.remaining_delta_v > 15:
    vessel.control.throttle = 1
while circ.remaining_delta_v > 0.1:
    vessel.control.throttle = 0.05

circ.remove()
vessel.control.throttle = 0

print('Circularization complete.')

**CSM Detach, Flip, Dock Maneuver**

This docking method depends entirely on this craft, and doesn't work generally. It'd be nice to have a general docking script that works for any two crafts.

In [ ]:
# set vessel to SAS normal and wait until oriented
print('Orienting to normal... ', end='')
vessel.control.sas_mode = conn.space_center.SASMode.normal
time.sleep(3)
ang_vel = vessel.angular_velocity(vessel.orbit.body.reference_frame)
oriented = all(ang_vel[i] < 0.01 for i in range(3))
while not oriented:
    ang_vel = vessel.angular_velocity(vessel.orbit.body.reference_frame)
    oriented = all(ang_vel[i] < 0.01 for i in range(3))
print('Done.')
    
# turn off RCS and undock CSM
vessel.control.rcs = False
time.sleep(1)
vessel.control.activate_next_stage()

# turn on RCS, thrust normal briefly, then stop
vessel.control.rcs = True
time.sleep(1)
vessel.control.forward = 1
time.sleep(0.5)
vessel.control.forward = 0
time.sleep(2)
vessel.control.forward = -1
time.sleep(0.5)
vessel.control.forward = 0

# turn off RCS
vessel.control.rcs = False
print('CSM sep.')

# control CSM from docking port
print('Docking maneuver... ', end='')
CSM_docking_port = vessel.parts.with_title('Clamp-O-Tron Docking Port')[0]
vessel.parts.controlling = CSM_docking_port

# target LM docking port
vessels = conn.space_center.vessels
for v in vessels:
    if v.name == 'Auto Mun Lander':
        LM = v
    elif v.name == 'Auto Mun':
        CSM = v

LM_docking_port = LM.parts.docking_ports[0] # DockingPort object
conn.space_center.target_docking_port = LM_docking_port
        
# set CSM SAS to target
vessel.control.sas_mode = conn.space_center.SASMode.target
time.sleep(1)

# switch to LM
conn.space_center.active_vessel = LM
time.sleep(1)

# control LM from docking port
LM_parts = LM.parts.all
for part in LM_parts:
    if part.name == 'dockingPort2':
        LM_dp = part  # Part object
LM.parts.controlling = LM_dp  # Requires Part, not DockingPort
time.sleep(1)

# target CSM docking port
CSM_dp = CSM.parts.docking_ports[0]
conn.space_center.target_docking_port = CSM_dp
time.sleep(1)

# set LM SAS to target
LM.control.sas_mode = conn.space_center.SASMode.target
time.sleep(1)

# switch to CSM
conn.space_center.active_vessel = CSM
time.sleep(1)
CSM.control.sas_mode = conn.space_center.SASMode.target
time.sleep(1)
vessel.control.rcs = True
time.sleep(1)

# thrust slightly in direction of target docking port, turn off RCS, wait until docked
# this requires some Patience & Luck (TM)
n_vessels = len(conn.space_center.vessels)
vessel.control.forward = 1
time.sleep(1.5)
vessel.control.forward = 0

# wait until docked
while len(conn.space_center.vessels) == n_vessels:
    time.sleep(1)
    
# turn off RCS
vessel.control.rcs = False
print('Done.')

**Trans-Munar Injection**

In [ ]:
print('Setting up Trans-Munar Injection.')

# redefine newly docked vessel
CSM_LM_S2 = conn.space_center.active_vessel

# control from LM (for orientation purposes)
for part in CSM_LM_S2.parts.all:
    if part.name == 'mk2LanderCabin.v2':
        CSM_LM_S2.parts.controlling = part

# set Mun as target
conn.space_center.target_body = mun
time.sleep(1)

# create TMI maneuver node (Hohmann Transfer)

# calculate dV required for maneuver
v_s_hi = calc_circ_orb_speed(r=CSM_LM_S2.orbit.apoapsis, M=mass_kerbin)
v_s_lo = calc_circ_orb_speed(r=CSM_LM_S2.orbit.periapsis, M=mass_kerbin)
v_s = (v_s_hi + v_s_lo) / 2

r_b = CSM_LM_S2.orbit.semi_major_axis
a_tmi = (100000 + 600000 + mun.orbit.apoapsis) / 2
v_pe = vis_viva(r=r_b, a=a_tmi, M=mass_kerbin)

dv = v_pe - v_s

# calculate time until maneuver node

# half-period of transfer orbit
p = orbital_period(a=a_tmi, M=mass_kerbin)  # period of one TMI orbit
p_2 = 0.5 * p  # time of one half TMI orbit

# arc distance Mun travels in time p_2
v_m = mun.orbit.speed  # Mun orbital velocity
s = v_m * p_2  # arc-length distance Mun travels

# angle between Mun current pos and Mun intercept pos
r_m_i = mun.position(kerbin_nrrf)
theta = s / np.linalg.norm(r_m_i)  # radians

# Mun position at intercept
x1, y1, z1 = r_m_i[0], r_m_i[1], r_m_i[2]
x2 = x1 * np.cos(theta) - z1 * np.sin(theta)
y2 = 0
z2 = x1 * np.sin(theta) + z1 * np.cos(theta)

r_m_f = (x2, y2, z2)

# find time to burn
r_s_b = [-1 * r_m_f[i] for i in range(len(r_m_f))]
r_s_i = CSM_LM_S2.position(kerbin_nrrf)
r_s_i = [r_s_i[0], 0, r_s_i[2]] # nullify inclination

x1, y1 = r_s_i[0], r_s_i[2]
x2, y2 = r_s_b[0], r_s_b[2]

theta2 = np.arctan2(x1*y2-y1*x2, x1*x2+y1*y2)
if theta2 < 0:
    theta2 += 2*np.pi

s_to_b = CSM_LM_S2.orbit.semi_major_axis * theta2
t_to_b = s_to_b / v_s

# create node
ut_b = conn.space_center.ut + t_to_b
tmi = CSM_LM_S2.control.add_node(ut=ut_b)
tmi.prograde = dv

# set SAS to maneuver and wait until oriented
print('Orienting for maneuver.')

CSM_LM_S2.control.sas_mode = conn.space_center.SASMode.maneuver
CSM_LM_S2.control.rcs = True
time.sleep(3)
ang_vel = CSM_LM_S2.angular_velocity(CSM_LM_S2.orbit.body.reference_frame)
oriented = all(ang_vel[i] < 0.1 for i in range(3))
while not oriented:
    ang_vel = CSM_LM_S2.angular_velocity(CSM_LM_S2.orbit.body.reference_frame)
    oriented = all(ang_vel[i] < 0.1 for i in range(3))

# time warp to maneuver
print('Warping to maneuver.')

burn_time = calc_burn_duration(CSM_LM_S2, dv)
conn.space_center.rails_warp_factor = 2
while conn.space_center.ut < (ut_b - (burn_time / 2) - 60):
    time.sleep(1)
    
conn.space_center.rails_warp_factor = 0
conn.space_center.camera.mode = conn.space_center.CameraMode.automatic
time.sleep(1)

print('Burn baby, burn.')

# execute maneuver
while conn.space_center.ut < (ut_b - (burn_time / 2)):
    time.sleep(0.1)

while tmi.remaining_delta_v > 15:
    CSM_LM_S2.control.throttle = 1
while tmi.remaining_delta_v > 0.1:
    CSM_LM_S2.control.throttle = 0.05

tmi.remove()
CSM_LM_S2.control.throttle = 0
CSM_LM_S2.control.rcs = False

print('TMI complete.')

**Outbound Trajectory Correction**

In [ ]:
time.sleep(3)

# separate from Stage 2
for part in CSM_LM_S2.parts.all:
    if part.tag == 'Decoupler.S2':
        part.decoupler.decouple()

# rename current vessel
CSM_LM = conn.space_center.active_vessel

print('S2 sep.')

for part in CSM_LM.parts.all:
    if part.name == 'engineLargeSkipper':
        # activate SM engine
        part.engine.active = True
    
    elif part.name == 'liquidEngine2-2.v2':
        # ensure LM engine deactivated
        part.engine.active = False
    
    elif part.name == 'mk1-3pod':
        # control from CM
        CSM_LM.parts.controlling = part
        
# create outbound trajectory correction node
print('Set up OTC node.')
ut_otc = conn.space_center.ut + 3000
otc = CSM_LM.control.add_node(ut=ut_otc)
otc.prograde = 12.5

# maneuver to node
CSM_LM.control.sas_mode = conn.space_center.SASMode.maneuver
time.sleep(3)
ang_vel = CSM_LM.angular_velocity(CSM_LM.orbit.body.reference_frame)
oriented = all(ang_vel[i] < 0.01 for i in range(3))
while not oriented:
    ang_vel = CSM_LM.angular_velocity(CSM_LM.orbit.body.reference_frame)
    oriented = all(ang_vel[i] < 0.01 for i in range(3))

# warp carefully to node
print('Warping to node... ', end='')
conn.space_center.rails_warp_factor = 4
while conn.space_center.ut < (ut_otc - 1000):
    time.sleep(0.1)
conn.space_center.rails_warp_factor = 3
while conn.space_center.ut < (ut_otc - 300):
    time.sleep(0.1)
conn.space_center.rails_warp_factor = 2
while conn.space_center.ut < (ut_otc - 100):
    time.sleep(0.1)
conn.space_center.rails_warp_factor = 1
while conn.space_center.ut < (ut_otc - 15):
    time.sleep(0.1)
conn.space_center.rails_warp_factor = 0
print('Done.') 

# execute node
print('Cleaning up Mun intercept... ', end='')

while conn.space_center.ut < (ut_otc - 5):
    time.sleep(0.1)

while otc.remaining_delta_v > 0.1:
    CSM_LM.control.throttle = 0.05

CSM_LM.control.throttle = 0
otc.remove()

vessel = CSM_LM

print('Done.')

**Circularize Lunar Orbit**

In [ ]:
# wait until in Mun SOI
print('Warping to Mun SOI... ', end='')
time.sleep(1.0)  # wait for clunkiness to clear
conn.space_center.rails_warp_factor = 5
while vessel.orbit.body.name != 'Mun':
    time.sleep(0.1)
time.sleep(1.0)  # wait for SOI change clunkiness to clear
conn.space_center.rails_warp_factor = 0
print('Done.')

# create circularization maneuver node
t_to_pe = vessel.orbit.time_to_periapsis
ut_pe = conn.space_center.ut + t_to_pe
speed_pe = v_pe_hyperbolic(M=mass_mun, a=vessel.orbit.semi_major_axis, \
                           e=vessel.orbit.eccentricity)
speed_circ = calc_circ_orb_speed(r=vessel.orbit.periapsis, M=mass_mun)
dV = -abs(speed_pe - speed_circ)  # minus to slow down
circ = vessel.control.add_node(ut=ut_pe, prograde=dV)

# warp to node
burn_duration = calc_burn_duration(vessel, dV)
warp_to_node(conn, vessel, circ, burn_duration)

# execute maneuver
print('Executing node... ', end='')
execute_node(conn, vessel, circ, burn_duration)
print('Done.')

**Crew Transfer**

TODO: can't figure out how to transfer crew automatically. For now, need to launch with two pilots, one in each command pod.

**Mun Landing**

In [ ]:
# fill lander fuel tanks from SM fuel tanks


# lander detach


# wait until passing over night->day meridian


# deorbit burn


# landing sequence